# Interacting with Transformers through HuggingFace's API

**Important** It is recommended that you execute this notebook on a GPU-enabled machine. You can do this from Google Colab as follows:

1. Visit https://colab.research.google.com/
2. Click on the GitHub tab
3. Paste the URL to the repository or notebook. You will need to select A100 GPU runtime. 

https://github.com/PacktPublishing/Building-Natural-Language-Pipelines


![](./images/colab.png)

![](./images/runtime.png)

Run the command below if you are running this from colab. 

You will need tup upload de requirements.txt in this repository in the same directory as your Google colab notebook.

In [ ]:
!pip install -r requirements.txt -q

Initializing model access and tokenizer.

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "bigscience/bloomz-3b"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)

Tokenization: Before feeding text to a model, you need to tokenize it. Using the above tokenizer, you can tokenize a sentence like this:

In [ ]:
inputs = tokenizer.encode("Translate to English: Je t’aime.", return_tensors="pt")

Model Inference: Once tokenized, you can feed the input tensors to the model to get predictions:

In [ ]:
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))

In [ ]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail")

## Preparing the data

https://huggingface.co/datasets/cnn_dailymail

## Preparing a model for fine-tuning

In [ ]:
from transformers import AutoModelForCausalLM

# huggingface hub model id
model_id =  "bigscience/bloomz-3b"

# load model from the hub
model = AutoModelForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map="auto")


Load data collator to take care of padding inputs and labels

Loading LoRA configuration

/Users/macpro/anaconda3/envs/llm-pipelines/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

# Define LoRA Config
lora_config = LoraConfig(
 r=16,
 lora_alpha=32,
 target_modules=["query_key_value"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.CAUSAL_LM
)
# prepare int-8 model for training
model = prepare_model_for_int8_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# trainable params: 4915200 || all params: 3007472640 || trainable%: 0.1634329082375293


In [ ]:
from transformers import DataCollatorForLanguageModeling

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer = tokenizer
)



In [ ]:
output_dir = "lora-bloomz-3b"

# Define training args
training_args = transformers.TrainingArguments(
    output_dir=output_dir,
	auto_find_batch_size=True,
    learning_rate=1e-3, # higher learning rate
    num_train_epochs=5,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=500,
    save_strategy="no",
    report_to="tensorboard",
)

# Create Trainer instance
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
)

model.config.use_cache = False  

In [ ]:
model_id="results-lora-bloomz-3b" 
hf_username = "your-username" 
peft_model_id = f'{model_id}/{hf_username}'
trainer.model.save_pretrained(peft_model_id) 
tokenizer.save_pretrained(peft_model_id) 